In [1]:
import sys
sys.path.insert(0, '/home/toepfer/Documents/Project_PhysNet3/KaiAsparagus')

# Reference Calculation on Local Workstations or Clusters

In [5]:
from asparagus.sample import Sampler
sampler = Sampler(
    config='smpl_nh3.json',
    sample_data_file='smpl_nh3.db',
    sample_systems='ref/meta_nh3.traj',
    sample_calculator='shell',
    sample_calculator_args = {
        'files': [
            'template/shell/run_orca.sh',
            'template/shell/run_orca.inp',
            'template/shell/run_orca.py',
            ],
        'files_replace': {
            '%xyz%': '$xyz',
            '%charge%': '$charge',
            '%multiplicity%': '$multiplicity',
            },
        'execute_file': 'template/shell/run_orca.sh',
        'charge': 0,
        'multiplicity': 1,
        'directory': 'shell',
        'result_properties': ['energy', 'forces', 'dipole']
        },
    sample_num_threads=4,
    sample_save_trajectory=True,
    )
#sampler.run()

The shell scripts will be executed by asparagures using ```bash run_orca.sh```

run_orca.sh
```
rm -f run_orca.gbw run_orca.ges
orca run_orca.inp > run_orca.out
python run_orca.py
```

run_orca.inp
```
! engrad RI PBE D3BJ def2-SVP def2/J TightSCF 
%pal nprocs 1 end 
*xyz %charge% %multiplicity%
%xyz%
*

```
becomes
```
! engrad RI PBE D3BJ def2-SVP def2/J TightSCF 
%pal nprocs 1 end 
*xyz 0 1
N     0.07351617  -0.00534278  -0.23603505 
H    -1.03994361  -0.03392835  -0.40634277 
H     0.31140533   0.88603402  -0.62595885 
H     0.48690189  -0.79495547  -0.78437965 
*
``

run_orca.py
```
import os
import re
import json

import numpy as np

from ase import units

# Orca output and result file path
output_file = "run_orca.out"
gradient_file = "run_orca.engrad"
result_file = "results.json"

def save_results(results, result_file):
    """
    Save result dictionary as json file
    """
    with open(result_file, 'w') as f:
        json.dump(results, f)

# If output file not written, save empty result dictionary
if not os.path.exists("run_orca.out"):
    save_results({}, result_file)
    exit()

# Initialize result dictionary
results = {}

# Read output file
with open(output_file, 'r') as f:
    flines = f.read()

# Read energy
re_energy = re.compile(r"FINAL SINGLE POINT ENERGY.*\n")
re_not_converged = re.compile(r"Wavefunction not fully converged")
found_line = re_energy.search(flines)
if found_line and not re_not_converged.search(found_line.group()):
    results['energy'] = float(found_line.group().split()[-1])*units.Hartree

# Read dipole
re_dipole = re.compile(r"Total Dipole Moment.*\n")
re_not_converged = re.compile(r"Wavefunction not fully converged")
found_line = re_dipole.search(flines)
if found_line and not re_not_converged.search(found_line.group()):
    results['dipole'] = list(np.array(
        found_line.group().split()[-3:], dtype=float)*units.Bohr)

# Read gradient file
with open(gradient_file, 'r') as f:
    flines = f.readlines()

# Read forces
getgrad = False
gradients = []
tempgrad = []
for i, line in enumerate(flines):
    if line.find('# The current gradient') >= 0:
        getgrad = True
        gradients = []
        tempgrad = []
        continue
    if getgrad and "#" not in line:
        grad = line.split()[-1]
        tempgrad.append(float(grad))
        if len(tempgrad) == 3:
            gradients.append(tempgrad)
            tempgrad = []
    if '# The at' in line:
        getgrad = False
results['forces'] = [
    list(-np.array(grad_i)*units.Hartree/units.Bohr)
    for grad_i in gradients]

# Save results
save_results(results, result_file)
```

with result .json file read by Asparagus
```
{"energy": -1535.4234625489537,
 "dipole": [-0.08876947707208435, 0.02660703014714993, -0.33175706861878895],
 "forces": [
   [-3.235397732194288, -1.474455583716518, 0.3764568818272469],
   [2.949547650701895, -0.22134084646210062, 0.3515452359394994],
   [0.6608213322874061, 0.8358449528315552, -0.7880407905321968],
   [-0.3749712507435909, 0.8599514773470633, 0.06003867276545045]]}
```